# Engine

In [1]:
from sqlalchemy import create_engine

"""
Does not connect to the database yet;
that happens only the first time it is asked
to perform a task against the database.
"""
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)

# No Automatic Commit

In [2]:
from sqlalchemy import text

# does not commit automatically
with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())

2025-11-13 11:19:23,348 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-13 11:19:23,349 INFO sqlalchemy.engine.Engine select 'hello world'
2025-11-13 11:19:23,350 INFO sqlalchemy.engine.Engine [generated in 0.00202s] ()
[('hello world',)]
2025-11-13 11:19:23,352 INFO sqlalchemy.engine.Engine ROLLBACK


# Commit as you go

In [3]:
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}],
    )

    # explicit commit is required
    conn.commit()

2025-11-13 11:19:23,458 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-13 11:19:23,460 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2025-11-13 11:19:23,462 INFO sqlalchemy.engine.Engine [generated in 0.00381s] ()
2025-11-13 11:19:23,465 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2025-11-13 11:19:23,467 INFO sqlalchemy.engine.Engine [generated in 0.00162s] [(1, 1), (2, 4)]
2025-11-13 11:19:23,468 INFO sqlalchemy.engine.Engine COMMIT


# Begin Once

In [4]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, {"x": 9, "y": 10}],
    )

2025-11-13 11:19:23,619 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-13 11:19:23,621 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2025-11-13 11:19:23,622 INFO sqlalchemy.engine.Engine [cached since 0.1567s ago] [(6, 8), (9, 10)]
2025-11-13 11:19:23,623 INFO sqlalchemy.engine.Engine COMMIT


# Print Rows

In [5]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))

    for row in result:
        print(f"x: {row.x}  y: {row.y}")

    print("----")

    for dict_row in result.mappings():
        x = dict_row["x"]
        y = dict_row["y"]
        print(f"x: {x}  y: {y}")

2025-11-13 11:19:23,723 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-13 11:19:23,724 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2025-11-13 11:19:23,725 INFO sqlalchemy.engine.Engine [generated in 0.00197s] ()
x: 1  y: 1
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
----
2025-11-13 11:19:23,727 INFO sqlalchemy.engine.Engine ROLLBACK


# Session

In [6]:
from sqlalchemy.orm import Session

stmt = text("SELECT x, y FROM some_table WHERE y > :y ORDER BY x, y")

with Session(engine) as session:
    result = session.execute(stmt, {"y": 6})
    for row in result:
        print(f"x: {row.x}  y: {row.y}")
    # I rollback automatically at the end of the block
    # internal connection is returned to the connection pool

2025-11-13 11:19:25,557 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-13 11:19:25,560 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ? ORDER BY x, y
2025-11-13 11:19:25,562 INFO sqlalchemy.engine.Engine [generated in 0.00187s] (6,)
x: 6  y: 8
x: 9  y: 10
2025-11-13 11:19:25,565 INFO sqlalchemy.engine.Engine ROLLBACK


In [7]:
with Session(engine) as session:
    result = session.execute(
        text("UPDATE some_table SET y=:y WHERE x=:x"),
        [{"x": 9, "y": 11}, {"x": 13, "y": 15}],
    )

    session.commit()

2025-11-13 11:19:26,784 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-13 11:19:26,786 INFO sqlalchemy.engine.Engine UPDATE some_table SET y=? WHERE x=?
2025-11-13 11:19:26,788 INFO sqlalchemy.engine.Engine [generated in 0.00154s] [(11, 9), (15, 13)]
2025-11-13 11:19:26,789 INFO sqlalchemy.engine.Engine COMMIT


# Database Metadata

In [8]:
from sqlalchemy import MetaData
# a dictionary that stores Table objects
metadata_obj = MetaData()

## Table

In [9]:
from sqlalchemy import Table, Column, Integer, String
user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

In [10]:
user_table.c.name

Column('name', String(length=30), table=<user_account>)

In [11]:
user_table.c.keys()

['id', 'name', 'fullname']

In [12]:
user_table.primary_key

PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))

In [13]:
from sqlalchemy import ForeignKey
address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False),
)

## Emit DDL

In [14]:
metadata_obj.create_all(engine)

2025-11-13 11:19:27,304 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-13 11:19:27,306 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2025-11-13 11:19:27,307 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-13 11:19:27,308 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2025-11-13 11:19:27,310 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-13 11:19:27,311 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2025-11-13 11:19:27,313 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-13 11:19:27,314 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2025-11-13 11:19:27,316 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-13 11:19:27,317 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2025-11-13 11:19:27,317 INFO sqlalchemy.engine.Engine [no key 0.00048s] ()
2025-11-13 11:19:27,319 INFO sqlalchemy.engine.Engine 
C

## Declarative Base

In [15]:
from sqlalchemy.orm import DeclarativeBase

class Base(DeclarativeBase):
    pass

In [16]:
Base.metadata

MetaData()

In [17]:
Base.registry

In [18]:
from typing import List
from typing import Optional
from sqlalchemy.orm import Mapped, mapped_column, relationship


class User(Base):
    __tablename__ = "user_account"

    # mapped_column generates a Column object
    id: Mapped[int] = mapped_column(primary_key=True)

    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    addresses: Mapped[List["Address"]] = relationship(back_populates="user")

    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"


class Address(Base):
    __tablename__ = "address"

    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id = mapped_column(ForeignKey("user_account.id"))
    user: Mapped[User] = relationship(back_populates="addresses")

    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

## Emit DDL

In [19]:
Base.metadata.create_all(engine)

2025-11-13 11:19:27,614 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-13 11:19:27,616 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2025-11-13 11:19:27,617 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-13 11:19:27,619 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2025-11-13 11:19:27,620 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-13 11:19:27,621 INFO sqlalchemy.engine.Engine COMMIT


# Insert

In [20]:
from sqlalchemy import insert
stmt = insert(user_table).values(name="spongebob", fullname="Spongebob Squarepants")
print(stmt)

INSERT INTO user_account (name, fullname) VALUES (:name, :fullname)


In [21]:
compiled = stmt.compile()
compiled.params

{'name': 'spongebob', 'fullname': 'Spongebob Squarepants'}

In [22]:
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

2025-11-13 11:19:28,019 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-13 11:19:28,021 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2025-11-13 11:19:28,022 INFO sqlalchemy.engine.Engine [generated in 0.00305s] ('spongebob', 'Spongebob Squarepants')
2025-11-13 11:19:28,024 INFO sqlalchemy.engine.Engine COMMIT


In [23]:
result.inserted_primary_key_rows

[(1,)]

In [24]:
with engine.connect() as conn:
    result = conn.execute(
        insert(user_table),
        [
            {"name": "sandy", "fullname": "Sandy Cheeks"},
            {"name": "patrick", "fullname": "Patrick Star"},
        ],
    )
    conn.commit()

2025-11-13 11:19:28,189 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-13 11:19:28,190 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2025-11-13 11:19:28,190 INFO sqlalchemy.engine.Engine [generated in 0.00198s] [('sandy', 'Sandy Cheeks'), ('patrick', 'Patrick Star')]
2025-11-13 11:19:28,191 INFO sqlalchemy.engine.Engine COMMIT


# Select

In [25]:
from sqlalchemy import select
stmt = select(user_table).where(user_table.c.name == "spongebob")
print(stmt)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


In [26]:
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(type(row))
        print(row)

2025-11-13 11:19:28,300 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-13 11:19:28,301 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2025-11-13 11:19:28,303 INFO sqlalchemy.engine.Engine [generated in 0.00362s] ('spongebob',)
<class 'sqlalchemy.engine.row.Row'>
(1, 'spongebob', 'Spongebob Squarepants')
2025-11-13 11:19:28,304 INFO sqlalchemy.engine.Engine ROLLBACK


## Row has Class Instances

In [27]:
stmt = select(User).where(User.name == "spongebob")
with Session(engine) as session:
    for row in session.execute(stmt):
        print(type(row)) # still Row()
        # but contains instance of class User
        print(row)

2025-11-13 11:19:28,415 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-13 11:19:28,429 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2025-11-13 11:19:28,431 INFO sqlalchemy.engine.Engine [generated in 0.00197s] ('spongebob',)
<class 'sqlalchemy.engine.row.Row'>
(User(id=1, name='spongebob', fullname='Spongebob Squarepants'),)
2025-11-13 11:19:28,434 INFO sqlalchemy.engine.Engine ROLLBACK


In [28]:
row[0]

User(id=1, name='spongebob', fullname='Spongebob Squarepants')

## Scalars

In [29]:
row = session.execute(select(User)).first()
print('---')
row[0]

2025-11-13 11:19:28,607 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-13 11:19:28,609 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2025-11-13 11:19:28,610 INFO sqlalchemy.engine.Engine [generated in 0.00133s] ()
---


User(id=1, name='spongebob', fullname='Spongebob Squarepants')

In [30]:
user = session.scalars(select(User)).first()
print('---')
user

2025-11-13 11:19:28,700 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2025-11-13 11:19:28,701 INFO sqlalchemy.engine.Engine [cached since 0.09157s ago] ()
---


User(id=1, name='spongebob', fullname='Spongebob Squarepants')

In [31]:
row = session.execute(select(User.name, User.fullname)).first()
row

2025-11-13 11:19:28,829 INFO sqlalchemy.engine.Engine SELECT user_account.name, user_account.fullname 
FROM user_account
2025-11-13 11:19:28,831 INFO sqlalchemy.engine.Engine [generated in 0.00200s] ()


('spongebob', 'Spongebob Squarepants')

## All

In [32]:
users = session.execute(select(User))
print('---')
users

2025-11-13 11:19:29,014 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2025-11-13 11:19:29,016 INFO sqlalchemy.engine.Engine [cached since 0.407s ago] ()
---


In [33]:
users = session.execute(select(User)).all()
print('---')
users

2025-11-13 11:19:29,185 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2025-11-13 11:19:29,186 INFO sqlalchemy.engine.Engine [cached since 0.5773s ago] ()
---


[(User(id=1, name='spongebob', fullname='Spongebob Squarepants'),),
 (User(id=2, name='sandy', fullname='Sandy Cheeks'),),
 (User(id=3, name='patrick', fullname='Patrick Star'),)]

In [34]:
users = session.scalars(select(User)).all()
print('---')
users

2025-11-13 11:19:29,331 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2025-11-13 11:19:29,332 INFO sqlalchemy.engine.Engine [cached since 0.7227s ago] ()
---


[User(id=1, name='spongebob', fullname='Spongebob Squarepants'),
 User(id=2, name='sandy', fullname='Sandy Cheeks'),
 User(id=3, name='patrick', fullname='Patrick Star')]

## Select Some Columns

In [35]:
print(select(user_table.c.name, user_table.c.fullname))

SELECT user_account.name, user_account.fullname 
FROM user_account


In [36]:
print(select(User.name, User.fullname))

SELECT user_account.name, user_account.fullname 
FROM user_account


## Where

In [37]:
print(user_table.c.name == "squidward")

user_account.name = :name_1


In [38]:
print(
    select(address_table.c.email_address).where(
        user_table.c.name == "squidward",
        address_table.c.user_id == user_table.c.id,
    )
)

SELECT address.email_address 
FROM address, user_account 
WHERE user_account.name = :name_1 AND address.user_id = user_account.id


In [39]:
from sqlalchemy import and_, or_

print(
    select(Address.email_address).where(
        and_(
            or_(User.name == "squidward", User.name == "sandy"),
            Address.user_id == User.id,
        )
    )
)

SELECT address.email_address 
FROM address, user_account 
WHERE (user_account.name = :name_1 OR user_account.name = :name_2) AND address.user_id = user_account.id


# ORM

## Flush Inserts

In [40]:
squidward = User(name="squidward", fullname="Squidward Tentacles")
krabs = User(name="ehkrabs", fullname="Eugene H. Krabs")

In [41]:
squidward.id

In [42]:
session = Session(engine)

In [43]:
session.add(squidward)
session.add(krabs)

In [44]:
squidward.id

In [45]:
session.new

IdentitySet([User(id=None, name='squidward', fullname='Squidward Tentacles'), User(id=None, name='ehkrabs', fullname='Eugene H. Krabs')])

In [46]:
"""
writes all pending changes to the database,
but does not commit the transaction.
"""
session.flush()


"""
SQLAlchemy will automatically flush():
- Before any SELECT query that might need to see your pending INSERTs/UPDATEs.
- Before a COMMIT.
"""

2025-11-13 11:19:30,155 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-13 11:19:30,157 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2025-11-13 11:19:30,158 INFO sqlalchemy.engine.Engine [generated in 0.00016s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('squidward', 'Squidward Tentacles')
2025-11-13 11:19:30,159 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2025-11-13 11:19:30,161 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('ehkrabs', 'Eugene H. Krabs')


'\nSQLAlchemy will automatically flush():\n- Before any SELECT query that might need to see your pending INSERTs/UPDATEs.\n- Before a COMMIT.\n'

In [47]:
# can sync primary key attributes before commit
squidward.id

4

## Session Identity Map

In [48]:
some_squidward = session.get(User, 4)
some_squidward

User(id=4, name='squidward', fullname='Squidward Tentacles')

In [49]:
# points to the same object as squidward
some_squidward is squidward

True

In [50]:
# stops tracking objects but session still open
session.commit()

2025-11-13 11:19:30,687 INFO sqlalchemy.engine.Engine COMMIT


In [51]:
# session.rollback()
session.close()

# Relationship

## back_populates syncs both Tables

In [52]:
u1 = User(name="pkrabs", fullname="Pearl Krabs")
u1

User(id=None, name='pkrabs', fullname='Pearl Krabs')

In [53]:
u1.addresses

[]

In [54]:
a1 = Address(email_address="pearl.krabs@gmail.com")
a1

Address(id=None, email_address='pearl.krabs@gmail.com')

In [55]:
u1.addresses.append(a1)
u1.addresses

[Address(id=None, email_address='pearl.krabs@gmail.com')]

In [56]:
a1.user

User(id=None, name='pkrabs', fullname='Pearl Krabs')

## Add to session

In [57]:
session = Session(engine)
session.add(u1)

In [58]:
u1 in session

True

In [59]:
a1 in session

True

## Commit

In [60]:
session.commit()

2025-11-13 11:36:37,773 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-13 11:36:37,776 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2025-11-13 11:36:37,777 INFO sqlalchemy.engine.Engine [generated in 0.00143s] ('pkrabs', 'Pearl Krabs')
2025-11-13 11:36:37,782 INFO sqlalchemy.engine.Engine INSERT INTO address (email_address, user_id) VALUES (?, ?)
2025-11-13 11:36:37,784 INFO sqlalchemy.engine.Engine [generated in 0.00217s] ('pearl.krabs@gmail.com', 6)
2025-11-13 11:36:37,786 INFO sqlalchemy.engine.Engine COMMIT


In [61]:
u1.id

6

## Lazy Load

In [62]:
u1.addresses

2025-11-13 11:46:46,462 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.email_address AS address_email_address, address.user_id AS address_user_id 
FROM address 
WHERE ? = address.user_id
2025-11-13 11:46:46,464 INFO sqlalchemy.engine.Engine [generated in 0.00197s] (6,)


[Address(id=1, email_address='pearl.krabs@gmail.com')]

In [63]:
# no new sql emitted again
u1.addresses

[Address(id=1, email_address='pearl.krabs@gmail.com')]

## Expire

In [64]:
session.expire(a1)

In [65]:
u1.addresses

2025-11-13 12:08:09,277 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.email_address AS address_email_address, address.user_id AS address_user_id 
FROM address 
WHERE address.id = ?
2025-11-13 12:08:09,278 INFO sqlalchemy.engine.Engine [cached since 1890s ago] (1,)


[Address(id=1, email_address='pearl.krabs@gmail.com')]

## Object State

In [66]:
from sqlalchemy import inspect
insp = inspect(u1)
insp.persistent

True

In [67]:
insp

## Session States

In [68]:
# pending objects recently added to the Session
session.new

IdentitySet([])

In [69]:
# persistent objects which currently have changes detected
# (this collection is now created on the fly each time the property is called)
session.dirty

IdentitySet([])

In [70]:
# persistent objects that have been marked as deleted via session.delete(obj)
session.deleted

IdentitySet([])

In [71]:
# dictionary of all persistent objects, keyed on their
# identity key
session.identity_map

# Cascade

In [ ]:
"""
In SQLAlchemy, cascades define how operations on a parent object
automatically affect its related child objects in relationships.

They tell the ORM what to do with related objects when you
do things like add(), delete(), or commit() on the parent.
"""

from sqlalchemy.orm import backref


class Order(Base):
    __tablename__ = "order"

    items = relationship("Item", cascade="all, delete-orphan")
    customer = relationship("User", cascade="save-update")


class Item(Base):
    __tablename__ = "item"

    """
    backref is equivalent to defining the same items relationship
    on the Order side,
    so this line duplicates what you already wrote manually in Order.items.
    """
    order = relationship(
        "Order", backref=backref("items", cascade="all, delete-orphan")
    )